# Statistics Challenge (Optional)

Use the `orders.csv` dataset in the same directory to complete this challenge.

**Background**:

There are exactly 100 sneaker shops on a sneaker retailing website, and each of these shops sells only one model of shoe. We want to do some analysis of the average order value (AOV). When we look at orders data over a 30 day window, we naively calculate an AOV of $3145.13. Given that we know these shops are selling sneakers, a relatively affordable item, something seems wrong with our analysis. 

**Questions**:

- What went wrong with this metric (AOV) and the current analysis in general. Support your answers with appropriate visualizations.

- Propose a new metric that better represents the behavior of the stores' customers. Why are these metrics better and what are their values?

- Using the `created_at` column as a proxy for the date and time of order creation, provide a brief analysis of the trends in the customers' purchasing behavior.

Show all of your work in this notebook.

### 1. What went wrong with this metric (AOV) and the current analysis in general. Support your answers with appropriate visualizations.

Most of the order values are much lower than AOV $3145.13, so this average value deviates too much from most of the orders, and it cannot reflect the general situation. 

We can use a boxplot to show the general distribution of data.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv('orders.csv')
sns.boxplot(x=df['order_value'])
plt.title('Boxplot of Order Value')
plt.show()


However, in this boxplot, the large order value is too large, so we can't see the main part of this boxplot. Therefore, I consider using the log of order value.

In [ ]:
import numpy as np

df = pd.read_csv('orders.csv')
df['log_order_value'] = np.log(df['order_value'])
sns.boxplot(x=df['log_order_value'])
plt.title('Boxplot of Log-transformed Order Value')
plt.show()


From this boxplot, we can see that there are some large values divating from other values a lot, which causes the average value to be larger than normal.

### 2. Propose a new metric that better represents the behavior of the stores' customers. Why are these metrics better and what are their values?

We can use the median value instead of the average value, as a few extreme large value will not affect the median value too much. Median value can reflect the general situation of customers' choices.

### 3. Using the `created_at` column as a proxy for the date and time of order creation, provide a brief analysis of the trends in the customers' purchasing behavior.

In [ ]:
import matplotlib.dates as mdates

df = pd.read_csv('orders.csv')
df['created_at'] = pd.to_datetime(df['created_at'])
df.set_index('created_at', inplace=True)
daily_orders = df.resample('D').apply({'order_id': 'count', 'order_value': 'sum'})
fig, ax1 = plt.subplots()
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d'))

color = 'tab:red'
ax1.set_xlabel('Date')
ax1.set_ylabel('Daily Orders', color=color)
ax1.plot(daily_orders.index, daily_orders['order_id'], color=color)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()
color = 'tab:blue'
ax2.set_ylabel('Daily Order Value', color=color)
ax2.plot(daily_orders.index, daily_orders['order_value'], color=color)
ax2.tick_params(axis='y', labelcolor=color)

fig.tight_layout()
plt.title('Daily Orders and Order Value')
plt.show()


This is a graph of the orders and total order value each day. We can see that the value each day is changing very fast.

In [ ]:
df = pd.read_csv('orders.csv')
df['created_at'] = pd.to_datetime(df['created_at'])
df['weekday'] = df['created_at'].dt.weekday
weekday_orders = df.groupby('weekday').agg({'order_id': 'count', 'order_value': 'sum'}).reset_index()
weekday_names = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']
x = np.arange(len(weekday_names))
width = 0.35
fig, ax1 = plt.subplots()

rects1 = ax1.bar(x - width / 2, weekday_orders['order_id'], width, color='tab:red')
ax1.set_xlabel('Weekday')
ax1.set_ylabel('Orders Count', color='tab:red')
ax1.set_xticks(x)
ax1.set_xticklabels(weekday_names)
ax1.tick_params(axis='y', labelcolor='tab:red')
ax2 = ax1.twinx()

rects2 = ax2.bar(x + width / 2, weekday_orders['order_value'], width, color='tab:blue')
ax2.set_ylabel('Total Order Value', color='tab:blue')
ax2.tick_params(axis='y', labelcolor='tab:blue')

fig.tight_layout()
plt.title('Weekday Orders and Order Value')
plt.show()


This is a graph of the orders and total order value in each week day. We can see that the total order value in Sunday is very small, and on Monday it is largest. The number of orders don't differ much in each week day.

In [ ]:
df = pd.read_csv('orders.csv')
df['created_at'] = pd.to_datetime(df['created_at'])
df['hour'] = df['created_at'].dt.hour
hourly_orders = df.groupby('hour').agg({'order_id': 'count', 'order_value': 'sum'}).reset_index()
fig, ax1 = plt.subplots(figsize=(10, 6))

ax1.plot(hourly_orders['hour'], hourly_orders['order_id'], color='tab:red', marker='o', label='Total Orders')
ax1.set_xlabel('Hour of the Day')
ax1.set_ylabel('Total Orders', color='tab:red')
ax1.tick_params(axis='y', labelcolor='tab:red')
ax1.set_ylim([0, hourly_orders['order_id'].max() + 10])

ax2 = ax1.twinx()
ax2.plot(hourly_orders['hour'], hourly_orders['order_value'], color='tab:blue', marker='x', label='Total Order Value')
ax2.set_ylabel('Total Order Value', color='tab:blue')
ax2.tick_params(axis='y', labelcolor='tab:blue')
ax2.set_ylim([0, hourly_orders['order_value'].max() + 1000000])

plt.title('Total Orders and Order Value by Hour')
ax1.grid(True)
plt.show()


This is a graph of the orders and total order value in each hour. We can see that the number of orders don't differ much in each hour. But between 4 am to 5 am, the total order value is much higher than all the other hours. 